In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import bs4
import requests
from tqdm import tqdm
import re
import urllib

## Url of products

In [2]:
header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
def get_links(no_pages ,http='https://www.amazon.in'):
    links = []
    links_dic = {}
    for page in tqdm(range(1 , no_pages+1)):
        req = urllib.request.Request(f"https://www.amazon.in/s?k=bags&page={page}&crid=2M096C61O4MLT&qid=1685030690&sprefix=ba%2Caps%2C283&ref=sr_pg_{page}",headers = header)
        req.add_header('User-agent', "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")
        content = urllib.request.urlopen(req).read()
        soup  = BeautifulSoup(content , "html.parser")
        tags = soup.find_all("a" ,class_ ='a-size-base a-link-normal s-no-hover s-underline-text s-underline-link-text s-link-style a-text-normal')
        l = []
        for t in tags:
            if t["href"].startswith("http"):
                links.append(t["href"])
                l.append(t["href"])
            else:
                links.append(http+t["href"])
                l.append(http+t["href"])
        links_dic[page] = l
    return links,links_dic

In [3]:
links,links_dic = get_links(20)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:43<00:00,  2.17s/it]


In [28]:
len(links)

450

## 450 product links are taken from 20 pages of website

## Information of  the product

In [237]:
header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
def get_info(links,links_dic ,http='https://www.amazon.in'):
    product_info = {}
    for page in tqdm(links_dic):
        for link in tqdm(links_dic[page]):
            info = {}
            req = urllib.request.Request(link)
            req.add_header('User-agent', "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")
            content = urllib.request.urlopen(req).read()
            soup  = BeautifulSoup(content , "html.parser")
            
            try:
                name = soup.find("span",id = "productTitle").text.strip()
            except:
                name = "unknown"
            try:
                price = soup.find(class_ = 'a-price-whole').text
            except:
                price = "unknown"
            try: 
                rate = soup.find(class_ = 'a-icon-alt').text.split(" ")[0]
                n_rate = soup.find("span" , id = 'acrCustomerReviewText' ,class_ = 'a-size-base').text
            except:
                rate = "0"
                n_rate = "0"
            
            info["product_name"] = name
            info["product_link"] = link
            info["product_price"] = price
            if len(n_rate) >0 :
                info["product_rating"] = rate
                info["no_rating"] = n_rate
            else:
                info["product_rating"] = 0
                info["no_rating"] = 0
            info["page_no"] = page
            
            if name not in product_info:
                product_info[name] = info
            else:
                product_info[name+'_1'] = info
        
    return product_info

In [238]:
%%time
product_info = get_info(links,links_dic)

  0%|                                                                                           | 0/22 [00:00<?, ?it/s]


 52%|██████████████████████████████████████████▉                                       | 11/21 [00:34<00:32,  3.22s/it]


 22%|██████████████████                                                                 | 5/23 [00:17<01:04,  3.59s/it]


100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [42:27<00:00, 127.37s/it]

Wall time: 42min 27s


In [259]:
data = pd.DataFrame(data = product_info)

In [255]:
pd.DataFrame(data = product_info).to_csv("product_info.csv")

In [33]:
datap = pd.read_csv('product_info.csv')
datap = datap.set_index(data2["Unnamed: 0"].values).drop("Unnamed: 0",axis=1)

In [39]:
dupli = []
for i in datap.columns:
    if i.endswith('_1'):
        dupli.append(i)
datap.drop(dupli,axis=1,inplace = True)

## Product info without duplicates

In [40]:
datap.T.info()

<class 'pandas.core.frame.DataFrame'>
Index: 245 entries, MOKOBARA Backpack 15.5" Inch Laptop Backpack For Women (16 litres ) Improved (2022 - Model) to F Gear Sniper Lite V2 30 Liters Black Laptop Backpack
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   product_name    245 non-null    object
 1   product_link    245 non-null    object
 2   product_price   245 non-null    object
 3   product_rating  245 non-null    object
 4   no_rating       245 non-null    object
 5   page_no         245 non-null    object
dtypes: object(6)
memory usage: 13.4+ KB


In [42]:
datap.T.head()

,product_name,product_link,product_price,product_rating,no_rating,page_no
"MOKOBARA Backpack 15.5"" Inch Laptop Backpack For Women (16 litres ) Improved (2022 - Model)","MOKOBARA Backpack 15.5"" Inch Laptop Backpack F...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,"4,490",4.3,168 ratings,1
MOKOBARA The Transit Backpack Laptop Bag for Men and Women,MOKOBARA The Transit Backpack Laptop Bag for M...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,"5,999",4.5,590 ratings,1
Wesley Milestone 2.0 Casual Waterproof Laptop Backpack/Office Bag/School Bag/College Bag/Business Bag/Travel Backpack (Dimensions:13x18 inches) (Compatible with 39.62cm(15.6inch laptop) 30 L (Blue & black),Wesley Milestone 2.0 Casual Waterproof Laptop ...,https://www.amazon.in/Wesley-Milestone-Waterpr...,598.,4.3,"10,923 ratings",1
American Tourister 32 Ltrs Black Casual Backpack (AMT FIZZ SCH BAG 02 - BLACK),American Tourister 32 Ltrs Black Casual Backpa...,https://www.amazon.in/American-Tourister-AMT-S...,"1,199.",4.0,"54,204 ratings",1
Skybags Brat Black 46 Cms Casual Backpack,Skybags Brat Black 46 Cms Casual Backpack,https://www.amazon.in/Skybags-Brat-Black-Casua...,669.,4.1,"4,547 ratings",1


In [43]:
datap.T.to_csv("product_info2.csv")
datap.T.to_csv("product_info3.csv",index = False)

## Among 450 products links 245 products links are unqiue.